<a href="https://colab.research.google.com/github/amandatz/computational-linear-algebra/blob/main/Atividade2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 2

In [1]:
using LinearAlgebra

In [2]:
using Printf

function print_matrix(M)
  for i in 1:size(M, 1)
    for j in 1:size(M, 2)
      @printf("%10.4f ", M[i, j])
    end
    println()
  end
end


print_matrix (generic function with 1 method)

## Questão 1

**Substituição regressiva**

In [13]:
function back_substitution(U, b)
  n = length(b)
  x = zeros(n)

  for i = n:-1:1
    x[i] = b[i]
    for j = i+1:n
      x[i] = x[i] - U[i,j] * x[j]
    end
    x[i] = x[i] / U[i,i]
  end

  return x
end

back_substitution (generic function with 1 method)

**Substituição direta**

In [14]:
function forward_substitution(L, b)
  n = length(b)
  x= zeros(n)

  for i=1:n
    x[i] = b[i]
    for j=1:i-1
      x[i] = x[i] - L[i,j]*x[j]
    end
    x[i] = x[i]/L[i,i]
  end

  return x
end

forward_substitution (generic function with 1 method)

In [5]:
n = 4
U = triu(rand(n, n))
L = tril(rand(n, n))
b = rand(n)

x_back = back_substitution(U, b)
x_forward = forward_substitution(L, b)

println("U = ")
print_matrix(U)
println("Solução (back-substitution):")
print_matrix(x_back)

println("\nL = ")
print_matrix(L)
println("Solução (forward-substitution):")
print_matrix(x_forward)


U = 
    0.3984     0.7366     0.1383     0.1343 
    0.0000     0.6482     0.4737     0.3036 
    0.0000     0.0000     0.7426     0.8089 
    0.0000     0.0000     0.0000     0.3691 
Solução (back-substitution):
   -0.5580 
    0.7292 
    0.3551 
    0.5403 

L = 
    0.6708     0.0000     0.0000     0.0000 
    0.9446     0.7536     0.0000     0.0000 
    0.8472     0.2038     0.4667     0.0000 
    0.8445     0.1911     0.0043     0.1028 
Solução (forward-substitution):
    0.6508 
    0.2524 
    0.2099 
   -3.8848 


## Questão 2

**Fatoração LU**

In [15]:
function flu(A)
  n = size(A, 1)
  L = Matrix{Float64}(I, n, n)
  U = copy(A)

  for k in 1:n-1
    for i in k+1:n
      L[i, k] = U[i, k] / U[k, k]
      U[i, k:n] -= L[i, k] * U[k, k:n]
    end
  end

  return L, U
end


flu (generic function with 1 method)

**Fatoração de Cholesky**

In [16]:
function fchol(A)
  n = size(A, 1)
  L = zeros(n, n)

  for i in 1:n
    for j in 1:i
      sum_k = dot(L[i, 1:j-1], L[j, 1:j-1])
        if i == j
          L[i, j] = sqrt(A[i, i] - sum_k)
        else
          L[i, j] = (A[i, j] - sum_k) / L[j, j]
        end
    end
  end

  return L
end

fchol (generic function with 1 method)

In [8]:
function solve_system(n)
  A = diagm(0 => fill(4.0, n),
            1 => fill(-1.0, n-1), -1 => fill(-1.0, n-1),
            2 => fill(-1.0, n-2), -2 => fill(-1.0, n-2))
  b = A[:, n]

  println("\n============================")
  println("n = $n")

  t_lu = @elapsed begin
    L, U = flu(A)# explicar pq admite
    y = forward_substitution(L,b)
    x_lu = back_substitution(U,y)
  end

  t_chol = @elapsed begin
    L = fchol(A)# explicar pq admite
    y = forward_substitution(L,b)
    x_chol = back_substitution(L',y)
  end

  x_true = zeros(n)
  x_true[end] = 1 # por que??? explicar
  err_lu = norm(x_lu - x_true) / norm(x_true)
  err_chol = norm(x_chol - x_true) / norm(x_true)

  println("== Fatoração LU ==")
  println("Tempo: ", t_lu, " segundos")
  println("Erro relativo: ", err_lu)

  println("\n== Fatoração de Cholesky ==")
  println("Tempo: ", t_chol, " segundos")
  println("Erro relativo: ", err_chol)
end

solve_system (generic function with 1 method)

In [9]:
for n in [10, 100, 200, 400]
    solve_system(n)
end


n = 10
== Fatoração LU ==
Tempo: 6.4449e-5 segundos
Erro relativo: 0.0

== Fatoração de Cholesky ==
Tempo: 3.7707e-5 segundos
Erro relativo: 2.4317759426983836e-16

n = 100
== Fatoração LU ==
Tempo: 0.013044541 segundos
Erro relativo: 0.0

== Fatoração de Cholesky ==
Tempo: 0.003510056 segundos
Erro relativo: 8.274363007295436e-16

n = 200
== Fatoração LU ==
Tempo: 0.211402081 segundos
Erro relativo: 0.0

== Fatoração de Cholesky ==
Tempo: 0.011230167 segundos
Erro relativo: 9.720503087223581e-16

n = 400
== Fatoração LU ==
Tempo: 0.404008713 segundos
Erro relativo: 0.0

== Fatoração de Cholesky ==
Tempo: 0.117709844 segundos
Erro relativo: 1.3701608414352226e-15


## Questão 3

## Questão 4

A matriz $A$ é diagonalmente dominante e, portanto, simétrica positiva definida. Nesse caso, pode-se utilizar a fatoração de Cholesky.

In [18]:
n=6

A = Matrix{Float64}(I, n, n)
A[1,2:n] .= 0.1
A[2:n,1] .= 0.1

L = fchol(A)
println("Matriz L:")
print_matrix(L)

Matriz L:
    1.0000     0.0000     0.0000     0.0000     0.0000     0.0000 
    0.1000     0.9950     0.0000     0.0000     0.0000     0.0000 
    0.1000    -0.0101     0.9949     0.0000     0.0000     0.0000 
    0.1000    -0.0101    -0.0102     0.9949     0.0000     0.0000 
    0.1000    -0.0101    -0.0102    -0.0103     0.9948     0.0000 
    0.1000    -0.0101    -0.0102    -0.0103    -0.0104     0.9948 


In [12]:
A

6×6 Matrix{Float64}:
 1.0  0.1  0.1  0.1  0.1  0.1
 0.1  1.0  0.0  0.0  0.0  0.0
 0.1  0.0  1.0  0.0  0.0  0.0
 0.1  0.0  0.0  1.0  0.0  0.0
 0.1  0.0  0.0  0.0  1.0  0.0
 0.1  0.0  0.0  0.0  0.0  1.0